In [1]:
from bf_multichannel import bf_multichannel
from tx_gnb_sim import tx_gnb_sim
import numpy as np
import matplotlib.pyplot as plt

# fc = 6.5kHz
n_UE = 1 # 2
# n_path = 2?
My_bf = bf_multichannel(6.5e3, 2, 5,n_UE)
# rx at (0,0)
# tx_1 at (5,5) and (7.5,5) with a wall at x=10
# after relection, tx_2 at (15,5) and (12.5,5)
My_bf.reflection_list = np.array([1,0.5]) # what does reflection_list accomplish?
My_bf.x_tx_list = np.array([5,15])
My_bf.y_tx_list = np.array([5,5])
#My_bf.reflection_list = np.array([1,0.5]) # what does reflection_list accomplish?
#My_bf.x_tx_list = np.array([5,15,7.5,12.5])
#My_bf.y_tx_list = np.array([5,5,5,5])

# rethink with this math

theta = My_bf.simulation()

My_ue = tx_gnb_sim(6.5e3, 1, 1, theta)
# do not assume independence
My_ue.reflection_list = np.array([1,0.5])
#My_ue.x_tx_list = np.array([5,15]) #shouldn't need to touch gnb location
#My_ue.y_tx_list = np.array([5,5]) #shouldn't need to touch gnb location
My_ue.x_rx_list = My_bf.x_tx_list
My_ue.y_rx_list = My_bf.y_tx_list

"""
plt.figure()
plt.plot(My_bf.snr_list, My_bf.mean_mse, 'o')
plt.xlabel('SNR(dB)')
plt.ylabel('mse(dB)')
plt.title(f'fc={My_bf.fc/1000:.1f}kHz')
plt.show()
"""

My_ue.simulation()

# plt mean square error
print(theta)
print(My_ue.MSE_SNR)
for i_UE in range(0,n_UE):
    plt.figure(i_UE)
    plt.plot(My_ue.snr_list, My_ue.mean_mse[:,i_UE], 'o')
    plt.xlabel('SNR(dB)')
    plt.ylabel('mse(dB)')
    plt.title(f'fc={My_bf.fc/1000:.1f}kHz, UE_ID={i_UE}')

plt.show()

SyntaxError: invalid syntax (tx_gnb_sim.py, line 162)